In [434]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats



In [435]:
np.random.seed(100)

In [436]:
# Trying to see how sensitive computed market shares are to the initialized parameters
# All the parameters of interest 
beta = np.array([2, -0.5, -0.3])
mu = 0.5
omega = 1

# alpha = -np.exp(mu + omega**2/2)

# The number of firms in the market
J = 10

# The number of product characteristics 
K = 2

# The number of consumers in the market 
N = 3

# Schockes per consumer 
v_p = np.random.normal(0, 1, size = N)

X1 = np.random.uniform(3, 6, size=J)
X2 = np.random.normal(3, 1, size=J)
all_X = np.column_stack((X1, X2))
X0 = np.ones(J)
X = np.column_stack((X0, all_X))

p = np.ones(J)
e = np.random.gumbel(size=N*J)
c = np.random.uniform(1, 2, size=J)




In [437]:
# Getting the summpation function over the 
# Making a killer function for calculating market shares 
def share(N_cons, J_prod, X_char, price, coef_car, mu, omega, e): 
    X_for_utility = np.repeat(X_char, N_cons, axis=0)
    price_r = np.reshape(price, (1, J_prod))
    alpha_i = np.reshape(-(np.exp(mu + omega*v_p)), (N_cons, 1))
    random_coeff = np.ravel((alpha_i*price_r).T)
    u = X_for_utility@coef_car+ random_coeff + e
    u_r = np.reshape(u, (J, N))
    sum_u = np.sum(np.exp(u_r))
    prob = np.exp(u_r)/(1 +sum_u)
    shares = np.sum(prob, axis=1)
    return(shares)


In [438]:
# X_for_utility = np.repeat(X, N, axis=0)
# price_r = np.reshape(p, (1, J))
# alpha_i = np.reshape(-(np.exp(mu + omega*v_p)), (N, 1))
# random_coeff = np.ravel((alpha_i*price_r).T)
# u = X_for_utility@beta + random_coeff + e
# print(u)
# u_r = np.reshape(u, (J, N))
# print(u_r)
# sum_u = np.sum(np.exp(u_r))
# prob = np.exp(u_r)/(1+sum_u)
# print(prob)
# shares = np.sum(prob, axis=1)


In [439]:
shares_my_market = share(N, J, X, p, beta, mu, omega, e)
print(shares_my_market)
print(sum(shares_my_market))

[0.71634878 0.23722698 0.01155923 0.00079005 0.00147784 0.00197255
 0.0045506  0.01257582 0.00073551 0.00520132]
0.9924386642626702


In [440]:
# Now gettting the correct Jacobian amtric with random coefficients and the stupid fucking error term 
# The random coff just to check how to rewrite the jacobian function 

J_ii = np.sum((-(np.exp(mu + omega*v_p)))* shares_my_market[0] - (-(np.exp(mu + omega*v_p)))*shares_my_market[0]**2)  
print(J_ii)

J_ij = np.sum(-(-(np.exp(mu + omega*v_p)))*shares_my_market[0]*shares_my_market[1])
print(J_ij)


-1.5914025465723973
1.330943022169499


In [441]:

def derivative(J_prod, mu, omega, shares):
    J = np.zeros((J_prod, J_prod))
    for i in range(J.shape[0]):
        for j in range(J.shape[1]):
            if i == j:
                # this is the own product elasticity
                J[i, j] = np.sum((-(np.exp(mu + omega*v_p)))* shares[i] - (-(np.exp(mu + omega*v_p)))*shares[i]**2)
            elif i!= j: 
                # this is the cross product elasticity
                J[i, j] = np.sum(-(-(np.exp(mu + omega*v_p)))*shares[i]*shares[j])
    return J




In [442]:
Jacobian = derivative(J, mu, omega, shares_my_market)
print(Jacobian)

[[-1.59140255e+00  1.33094302e+00  6.48521321e-02  4.43249135e-03
   8.29130791e-03  1.10668426e-02  2.55307638e-02  7.05556295e-02
   4.12651176e-03  2.91815746e-02]
 [ 1.33094302e+00 -1.41719713e+00  2.14765149e-02  1.46786949e-03
   2.74576011e-03  3.66490972e-03  8.45480032e-03  2.33652923e-02
   1.36654090e-03  9.66380748e-03]
 [ 6.48521321e-02  2.14765149e-02 -8.94850293e-02  7.15240734e-05
   1.33791150e-04  1.78578050e-04  4.11972427e-04  1.13850781e-03
   6.65866905e-05  4.70883057e-04]
 [ 4.43249135e-03  1.46786949e-03  7.15240734e-05 -6.18272776e-03
   9.14431178e-06  1.22053915e-05  2.81573506e-05  7.78143425e-05
   4.55104436e-06  3.21837542e-05]
 [ 8.29130791e-03  2.74576011e-03  1.33791150e-04  9.14431178e-06
  -1.15572957e-02  2.28311013e-05  5.26704386e-05  1.45557571e-04
   8.51307021e-06  6.02021289e-05]
 [ 1.10668426e-02  3.66490972e-03  1.78578050e-04  1.22053915e-05
   2.28311013e-05 -1.54184847e-02  7.03019910e-05  1.94283307e-04
   1.13628403e-05  8.03549322e-05

In [443]:
def root_objective(price, N_cons, J_prod, X_char, coef_car, mu, omega, e, c):
    X_for_utility = np.repeat(X_char, N_cons, axis=0)
    price_r = np.reshape(price, (1, J_prod))
    alpha_i = np.reshape(-(np.exp(mu + omega*v_p)), (N_cons, 1))
    random_coeff = np.ravel((alpha_i*price_r).T)
    u = X_for_utility@coef_car+ random_coeff + e
    u_r = np.reshape(u, (J, N))
    sum_u = np.sum(np.exp(u_r))
    prob = np.exp(u_r)/(1 +sum_u)
    shares = np.sum(prob, axis=1)
    Jacobian = derivative(J_prod, mu, omega, shares) 
    optim = np.matmul(np.transpose(Jacobian), (price - c)) + shares
    return optim

In [444]:
res1 = scipy.optimize.root(root_objective, p, args=(N, J, X, beta, mu, omega, e, c),
                            method='broyden2')
optimal_price = res1.x
print(optimal_price)
for i in range(0, J):
    if optimal_price[i] < c[i]:
        optimal_price = c
print(optimal_price)
print(c)

[5.66004004 5.71965337 5.52682235 5.98813585 5.85759342 6.07500167
 6.11186496 5.62457224 6.41522077 6.42833031]
[5.66004004 5.71965337 5.52682235 5.98813585 5.85759342 6.07500167
 6.11186496 5.62457224 6.41522077 6.42833031]
[1.17808099 1.23769421 1.04486228 1.50543143 1.37625245 1.5928054
 1.62994188 1.14260031 1.9338413  1.94637988]
